In [25]:
import requests
import json
import time
import logging

In [26]:
import requests, json

url = "https://albanyny-energovpub.tylerhost.net/apps/selfservice/api/energov/codecases/568bb576-c44d-46f2-807d-5166d1e0e5b3"

headers = {
    "Accept": "application/json, text/plain, */*",
    "Referer": "https://albanyny-energovpub.tylerhost.net/Apps/SelfService",
    "User-Agent": "Mozilla/5.0",
    "tenantId": "1",
    "tenantName": "AlbanyNY",
    "Tyler-TenantUrl": "AlbanyNY",
    "Tyler-Tenant-Culture": "en-US"
}

resp = requests.get(url, headers=headers)
print(resp.status_code)

try:
    data = resp.json()
    print(json.dumps(data, indent=2))
except Exception:
    print(resp.text[:1000])


200
{
  "Result": {
    "CodeCaseId": "568bb576-c44d-46f2-807d-5166d1e0e5b3",
    "CaseNumber": "CODE25-276453",
    "CodeCaseType": "CODE ENFORCEMENT",
    "StatusName": "Closed",
    "OpenedDate": "2025-07-30T15:06:43Z",
    "ClosedDate": "2025-08-18T12:42:07.92Z",
    "Description": "No ROPS",
    "ProjectName": null,
    "DistrictName": "Albany",
    "MainAddress": "171 SYCAMORE ST \r\nALBANY, NY 12209",
    "MainParcelNumber": "75.42-1-35",
    "AssignedTo": "Giovannone, Vincent",
    "CodeCaseTypeId": "48e1603a-4b28-40be-89bc-61793ed7241b",
    "LayoutId": "7b938b95-9670-4beb-b456-7fdfa4bcda90",
    "OnlineLayoutId": "1418a1cc-c4c0-1480-d201-04af28b6056c",
    "ActiveHolds": false,
    "UnPaidInvoicedFees": false,
    "UnPaidFees": false,
    "HoldExist": false,
    "IsLoggedIn": false,
    "AllowPayNow": false,
    "HasInvoiceIssue": false,
    "UnpaidInvoiceIds": [],
    "HasAuthorizedContact": false,
    "ProjectId": null
  },
  "Success": true,
  "ErrorMessage": "",
  "Valida

In [30]:
url = "https://albanyny-energovpub.tylerhost.net/apps/selfservice/api/energov/entity/violations/search"

headers = {
    "Accept": "application/json, text/plain, */*",
    "Referer": "https://albanyny-energovpub.tylerhost.net/Apps/SelfService",
    "User-Agent": "Mozilla/5.0",
    "tenantId": "1",
    "tenantName": "AlbanyNY",
    "Tyler-TenantUrl": "AlbanyNY",
    "Tyler-Tenant-Culture": "en-US"
}

resp = requests.get(url, headers=headers)
print(resp.status_code)

try:
    data = resp.json()
    print(json.dumps(data, indent=2))
except Exception:
    print(resp.text[:1000])


405
{
  "Message": "The requested resource does not support http method 'GET'."
}


In [31]:
import requests

url = "https://albanyny-energovpub.tylerhost.net/apps/selfservice/api/energov/entity/violations/search"
headers = {
    "Content-Type": "application/json",
}

payload = {
    "pageNumber": 1,
    "pageSize": 100,
    "sortField": "CaseNumber",
    "sortDirection": "asc",
    "searchText": "",  # you can filter by address, case number, etc.
    "filters": []      # optional filters; can leave empty
}

response = requests.post(url, json=payload, headers=headers)

print(response.status_code)
print(response.text)


200
{"PageCount":0,"TotalFound":0,"CacheKey":"","Result":null,"Success":false,"ErrorMessage":"Cannot find tenant information.","ValidationErrorMessage":"","ConcurrencyErrorMessage":"","StatusCode":200,"BrokenRules":[]}


In [32]:
import requests
import csv
import time

url = "https://albanyny-energovpub.tylerhost.net/apps/selfservice/api/energov/entity/violations/search"
headers = {
    "Content-Type": "application/json",
}

# Define the search filter for date range (if supported by Albany's EnerGov instance)
# Note: Some EnerGov sites ignore filters and just use searchText.
# We'll keep it empty here to fetch everything, but paginate through all results.
payload_template = {
    "pageNumber": 1,
    "pageSize": 100,
    "sortField": "CaseNumber",
    "sortDirection": "asc",
    "searchText": "",  # leave blank to get everything
    "filters": []      # could add filters here if Albany supports them
}

all_cases = []
page = 1

while True:
    payload = payload_template.copy()
    payload["pageNumber"] = page
    
    response = requests.post(url, json=payload, headers=headers)
    if response.status_code != 200:
        print(f"Error on page {page}: {response.status_code}")
        print(response.text)
        break

    data = response.json()
    result = data.get("Result")
    if not result or not result.get("Items"):
        print("No more items found. Stopping")
        break

    items = result["Items"]
    all_cases.extend(items)
    print(f"Fetched {len(items)} cases from page {page}")
    page += 1

    # Sleep to avoid overwhelming the server
    time.sleep(1)

print(f"\nTotal cases fetched: {len(all_cases)}")

No more items found. Stopping

Total cases fetched: 0


In [46]:
import requests
import time

url = "https://albanyny-energovpub.tylerhost.net/apps/selfservice/api/energov/entity/violations/search"
headers = {
    "Content-Type": "application/json",
}

payload = {
    "pageNumber": 1,
    "pageSize": 100,
    "sortField": "CaseNumber",
    "sortDirection": "asc",
    "searchText": " ",  # use a single letter to ensure matches
    "filters": [],
    "entityType": "CodeEnforcementCase",  # sometimes required
}

response = requests.post(url, json=payload, headers=headers)

print(response.status_code)
print(response.text[:1000])  # print first 1000 chars to preview


200
{"PageCount":0,"TotalFound":0,"CacheKey":"","Result":null,"Success":false,"ErrorMessage":"Cannot find tenant information.","ValidationErrorMessage":"","ConcurrencyErrorMessage":"","StatusCode":200,"BrokenRules":[]}


In [36]:
import requests
import json

# Step 1: Get Albany's tenant information
tenant_url = "https://albanyny-energovpub.tylerhost.net/apps/selfservice/api/energov/entity/violations/searchs"
tenant_resp = requests.get(tenant_url)
print("Tenant request status:", tenant_resp.status_code)

tenant_data = tenant_resp.json()
print(json.dumps(tenant_data, indent=2)[:1000])  # preview tenant info

# Extract tenant ID if present
tenant_id = None
if "Result" in tenant_data and tenant_data["Result"]:
    tenant_id = tenant_data["Result"][0].get("Id")

print("\nTenant ID:", tenant_id)

# Step 2: Only proceed if tenant_id found
if tenant_id:
    search_url = "https://albanyny-energovpub.tylerhost.net/apps/selfservice/api/energov/entity/violations/search"
    headers = {"Content-Type": "application/json"}
    
    payload = {
        "pageNumber": 1,
        "pageSize": 100,
        "sortField": "CaseNumber",
        "sortDirection": "asc",
        "searchText": "a",      # broad search to ensure matches
        "filters": [],
        "tenantId": tenant_id   # attach tenant context
    }

    search_resp = requests.post(search_url, json=payload, headers=headers)
    print("\nSearch request status:", search_resp.status_code)
    print(search_resp.text[:1000])  # preview results

else:
    print("\n❌ No tenant ID found — Albany’s portal may restrict public API access.")


Tenant request status: 404


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [37]:
pip install playwright

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 MB 50.1 MB/s  0:00:00m0:00:0100:01
  Attempting uninstall: greenlet
    Found existing installation: greenlet 1.1.1
    Uninstalling greenlet-1.1.1:
      Successfully uninstalled greenlet-1.1.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [playwright]3 [playwright]
Note: you may need to restart the kernel to use updated packages.


In [40]:
playwright install

SyntaxError: invalid syntax (938024582.py, line 1)

In [47]:
import asyncio
from playwright.async_api import async_playwright
import csv

output_file = "albany_code_violations.csv"

async def scrape_violations():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)  # see browser for debugging
        page = await browser.new_page()

        # Go directly to Code Cases page
        await page.goto("https://albanyny-energovpub.tylerhost.net/SelfService/#/search/codeCases", timeout=60000)
        await asyncio.sleep(5)  # wait for JS to render results

        # Wait for result rows
        await page.wait_for_selector(".result-item", timeout=60000)

        rows = await page.locator(".result-item").all()
        print(f"Found {len(rows)} rows on first page")

        cases = []
        for row in rows:
            case_number = await row.locator(".case-number").text_content()
            address = await row.locator(".address").text_content()
            cases.append({
                "CaseNumber": case_number.strip(),
                "Address": address.strip()
            })

        # Save to CSV
        with open(output_file, "w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=["CaseNumber", "Address"])
            writer.writeheader()
            writer.writerows(cases)

        await browser.close()
        print(f"✅ Saved {len(cases)} cases to {output_file}")

await scrape_violations()


ERROR:asyncio:Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed


TimeoutError: Page.wait_for_selector: Timeout 60000ms exceeded.
Call log:
  - waiting for locator(".result-item") to be visible


In [ ]:
pip install nest_asyncio

In [48]:
import nest_asyncio
nest_asyncio.apply()


In [49]:
import asyncio
from playwright.async_api import async_playwright
import csv
import nest_asyncio

nest_asyncio.apply()  # important in Jupyter

output_file = "albany_code_violations.csv"

async def scrape_violations():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        page = await browser.new_page()

        await page.goto("https://albanyny-energovpub.tylerhost.net/SelfService/#/search/codeCases", timeout=60000)
        await page.wait_for_selector(".result-item", timeout=60000)

        rows = await page.locator(".result-item").all()
        print(f"Found {len(rows)} rows on first page")

        cases = []
        for row in rows:
            case_number = await row.locator(".case-number").text_content()
            address = await row.locator(".address").text_content()
            cases.append({"CaseNumber": case_number.strip(), "Address": address.strip()})

        # Save to CSV
        with open(output_file, "w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=["CaseNumber", "Address"])
            writer.writeheader()
            writer.writerows(cases)

        await browser.close()
        print(f"✅ Saved {len(cases)} cases to {output_file}")

# Run in Jupyter
asyncio.run(scrape_violations())


TargetClosedError: Page.wait_for_selector: Target page, context or browser has been closed
Call log:
  - waiting for locator(".result-item") to be visible
